# Part 2 : Dashboard for surfers ! 
---
Autors: MISSONGO Aimé Blanchard & Hippolyte SODJINOU

## 1) Imports

In [ ]:
# Dependance
#! pip install -r requirements.txt

In [127]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from collections import defaultdict

## 2) Extract the data

In [205]:
def web_scap(url):
    response  = requests.get(url)
    response.raise_for_status()
    print("Connection OK")
    
    soup = BeautifulSoup(response.content, "html.parser")
    tables = soup.find_all("div", class_="forecast-tab")
    lines = soup.find_all('div', class_='line')
    # It is this list of dictionaries that will contain everything
    all_data = []

    for table in tables:
        # 1. Day recovery (e.g., "Friday, January 9")
        title_div = table.find("div", class_=re.compile("title"))
        day = title_div.get_text(strip=True) if title_div else "Inconnu"

        # 2. Retrieving data rows (skipping the header)
        lines = table.find_all("div", class_="line")[1:]

        for line in lines:
            cells = line.find_all("div", class_=re.compile("cell"))
            if len(cells) < 5:
                continue

            # Time extraction
            hour = cells[0].get_text(strip=True)

            # Wave extraction
            wave = cells[2].get_text(strip=True)

            # Extraction of wind direction (via the image's alt attribute)
            direction_div = line.find("div", class_=re.compile(r"wind.*img"))
            direction = None
            if direction_div:
                img = direction_div.find("img")
                if img:
                    direction = img.get("alt")

            # Wind speed extraction
            speed_div = line.find("div", class_=re.compile("large-bis-bis"))
            speed = speed_div.get_text(strip=True) if speed_div else "N/C"

            # --- ASSOCIATION: We are creating a dictionary for this line ---
            row = {
                "Jour": day,
                "Heure": hour,
                "Taille Vagues": wave,
                "Vitesse Vent(km/h)": speed,
                "Direction Vent": direction
            }
            
            # We add this line to our global list
            all_data.append(row)

    # --- CONVERSION TO DATAFRAME ---
    df = pd.DataFrame(all_data)
    
    return df

In [206]:
url = "https://www.surf-report.com/meteo-surf/lacanau-s1043.html"

data = web_scap(url)

Connection OK


In [207]:
# Display the results
#print(data.head())

# Save in csv
data.to_csv("meteo_surf.csv", index=False, encoding='utf-8-sig')

In [209]:
# data.head(20)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Jour                42 non-null     object
 1   Heure               42 non-null     object
 2   Taille Vagues       42 non-null     object
 3   Vitesse Vent(km/h)  42 non-null     object
 4   Direction Vent      42 non-null     object
dtypes: object(5)
memory usage: 1.8+ KB


In [210]:
data.head()

,Jour,Heure,Taille Vagues,Vitesse Vent(km/h),Direction Vent
0,Vendredi 9 Janvier,06:00,5.0-7.6,51,Orientation vent Ouest Nord Ouest
1,Vendredi 9 Janvier,09:00,5.3-8.0,54,Orientation vent Ouest
2,Vendredi 9 Janvier,12:00,5.6-8.4,60,Orientation vent Ouest
3,Vendredi 9 Janvier,15:00,5.5-8.2,56,Orientation vent Ouest Nord Ouest
4,Vendredi 9 Janvier,18:00,5.1-7.6,49,Orientation vent Nord Ouest
